# **Football Data Analysis: Pass / Shot Analysis & xGoals Modeling**

*Notebook summary*:
- **Feature Engineering**: Calculation of events' distances and angles
- **Pass Analysis**: Identification of elite passers based on accuracy, forward rate and final third penetration
- **Shot Analysis**: Deep dive into how distance, angle and body part affect goal probability
- **xGoals Model**: 
    - Logistic regression to predict shot outcomes
    - Application of the model on the Bundesliga to calculate an "expected" Bundesliga table based on the predicted goal-probability

*Possible extensions*:
- Comparison of xGoals and actual goals per player
- Deduction of expected goal-assists
- Evaluation of xConcededGoals per goalkeeper
- Comparison of pre- and post-shot xGoals-model (i.e. with / without considering where the shots were aimed) 

## **Import of libraries and data**

In [ ]:
# Standard libraries imports
from itertools import chain
import json
import math

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

# Machine learning
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

# Assets, constants, utils and visualizations from custom modules
import config as co
import utils as ut
import visualizations as vis

In [ ]:
dfs_events = {}
dfs_matches = {}
events = {}
matches = {}

competitions = [
    'england', 'european_championship', 'france',
    'germany', 'italy', 'spain', 'world_cup'
]

for competition in competitions:
    with open(
        f'data/events/events_{competition}.json',
        encoding='utf-8'
    ) as json_file:
        events[competition] = json.load(json_file)
        dfs_events[competition] = pd.DataFrame(events[competition])
        dfs_events[competition]['competition'] = competition
        print(
            f'{competition}: '
            f'{dfs_events[competition].shape[0]} events loaded.'
        )

    with open(
        f'data/matches/matches_{competition}.json',
        encoding='utf-8'
    ) as json_file:
        matches[competition] = json.load(json_file)
        dfs_matches[competition] = pd.DataFrame(matches[competition])
        dfs_matches[competition]['competition'] = competition
        print(
            f'{competition}: '
            f'{dfs_matches[competition].shape[0]} matches loaded.'
        )

df_events = pd.concat(
    [pd.DataFrame(dfs_events[competition]) for competition in competitions]
).reset_index()

df_matches = pd.concat(
    [pd.DataFrame(dfs_matches[competition]) for competition in competitions]
)
df_matches.set_index('wyId', inplace=True)

In [ ]:
df_tag_labels = pd.read_csv('data/mappings/tags2name.csv')
df_tag_labels.set_index('Tag', inplace=True)
df_event_labels = pd.read_csv('data/mappings/eventid2name.csv')

In [ ]:
with open('data/players.json', encoding='utf-8') as json_file:
    players = json.load(json_file)

df_players = pd.json_normalize(players)
df_players.set_index('wyId', inplace=True)

In [ ]:
with open('data/teams.json', encoding='utf-8') as json_file:
    teams_raw = json_file.read().encode().decode('unicode_escape')
    teams = json.loads(teams_raw)

df_teams = pd.DataFrame(teams)
df_teams.set_index('wyId', inplace=True)

## **Basic Cleaning and processing**

Some events' playerId is missig and therefore cannot be matched with a player from the database. Let's drop those entries.

In [ ]:
missing_player_id = df_events[df_events['playerId'] == 0].index
df_events.drop(missing_player_id, inplace=True)
print(
    f'{len(missing_player_id)} events with unrecorded playerId removed.'
)

We can simplify the 'tags'-column by flattening and unpacking the dictionaries to a list.

In [ ]:
df_events['tags'] = df_events['tags'].apply(
    lambda x: list(chain.from_iterable(d.values() for d in x))
)

The 'positions'-column can be split up for easier access.

In [ ]:
df_events['xPosStart'] = df_events['positions'].apply(
    lambda x: x[0]['x']
)

df_events['yPosStart'] = df_events['positions'].apply(
    lambda x: x[0]['y']
)

df_events['xPosEnd'] = df_events['positions'].apply(
    lambda x: x[1]['x'] if len(x) == 2 else x[0]['x']
)

df_events['yPosEnd'] = df_events['positions'].apply(
    lambda x: x[1]['y'] if len(x) == 2 else x[0]['y']
)

df_events.drop(
    columns=['positions'],
    inplace=True
)

## **Feature engineering** - Calculating distances

We can now calculate distance between start- and end-position.

In [ ]:
df_events['distance'] = df_events.apply(
    lambda row: np.linalg.norm(
        (
            (row['xPosEnd'] - row['xPosStart']) * co.PITCH_LENGTH / 100,
            (row['yPosEnd'] - row['yPosStart']) * co.PITCH_WIDTH / 100
        )
    ),
    axis=1
)

Some of the recorded events have a distance of 0m. Let's drop them, as they would cause problems when calculating passing-angles.

In [ ]:
events_with_zero_distance = df_events[df_events['distance'] == 0].index
df_events.drop(events_with_zero_distance, inplace=True)

print(
    f'{len(events_with_zero_distance)} events with zero distance removed.'
)

With some helper-functions we can calulate the shortest distance to the goal before and after the event.

In [ ]:
def y_closest_goal(y: float) -> float:
    if y < co.Y_LEFT_POST:
        y_closest = co.Y_LEFT_POST
    elif y > co.Y_RIGHT_POST:
        y_closest = co.Y_RIGHT_POST
    else:
        y_closest = y
    return y_closest

def distance_to_goal(x: float, y: float) -> float:
    vector_to_goal = (
        (100 - x) * co.PITCH_LENGTH / 100,
        (y_closest_goal(y) - y) * co.PITCH_WIDTH / 100
    )
    return np.linalg.norm(vector_to_goal)

In [ ]:
df_events['goalDistanceStart'] = df_events.apply(
    lambda row: distance_to_goal(row['xPosStart'], row['yPosStart']),
    axis=1
)

df_events['goalDistanceEnd'] = df_events.apply(
    lambda row: distance_to_goal(row['xPosEnd'], row['yPosEnd']),
    axis=1
)

df_events['goalDistanceCovered'] = (
    df_events['goalDistanceEnd'] - df_events['goalDistanceStart']
)

Let's tag events that started outside and ended inside the final third. This will be an interesting feature when analysing passes later.

In [ ]:
df_events['intoFinalThird'] = (
    (df_events['xPosStart'] < 66.67) &
    (df_events['xPosEnd'] >= 66.67)
)

##  **Feature engineering** - Calculating Angles

The events' angles can be calculated by their start- and end-positions. This will be an important feature for the pass-analysis as well.

In [ ]:
def angle(u: tuple, v: tuple) -> float:
    dot_prod = np.dot(u, v)
    u_norm = np.linalg.norm(u)
    v_norm = np.linalg.norm(v)
    return math.degrees(
        math.acos(dot_prod / (u_norm * v_norm))
    )

In [ ]:
df_events['angle'] = df_events.apply(
    lambda row: angle(
        (
            (row['xPosEnd'] - row['xPosStart']) * co.PITCH_LENGTH / 100,
            (row['yPosEnd'] - row['yPosStart']) * co.PITCH_WIDTH / 100
        ),
        (1, 0)
    ),
    axis=1
)

We can group together the angles into different categories.

In [ ]:
bins = [0, 22.5, 67.5, 112.5, 157.5, 180]

labels = [
    'vertical forward',
    'diagonal forward',
    'horizontal',
    'diagonal backward',
    'vertical backward'
]

df_events['angleCategory'] = pd.cut(
    df_events['angle'],
    bins=bins,
    labels=labels,
    include_lowest=True,
    ordered=False
)

We can also get each event's angle to the goal before and after the event by calculating the angle of the vector from the event's position to the closest point of the goal. This will be an important feature when inspecting shots.

In [ ]:
def angle_to_goal(x: float, y: float) -> float:
    vector_to_goal = (
        (100 - x) * co.PITCH_LENGTH / 100,
        (y_closest_goal(y) - y) * co.PITCH_WIDTH / 100
    )
    return angle(
        vector_to_goal,
        (1, 0)
    ) if not vector_to_goal == (0, 0) else 0

In [ ]:
df_events['goalAngleStart'] = df_events.apply(
    lambda row: angle_to_goal(row['xPosStart'], row['yPosStart']),
    axis=1
)

df_events['goalAngleEnd'] = df_events.apply(
    lambda row: angle_to_goal(row['xPosEnd'], row['yPosEnd']),
    axis=1
)

As an alternative approach of defining a positions angle to the goal, we define a position's 'shot-window' as the angle between the vectors from the event's position to each of the posts.

In [ ]:
def shot_window(x: float, y: float) -> float:
    left = (
        (100 - x) * co.PITCH_LENGTH / 100,
        (co.Y_LEFT_POST - y) * co.PITCH_WIDTH / 100
    )

    right = (
        (100 - x) * co.PITCH_LENGTH / 100,
        (co.Y_RIGHT_POST - y) * co.PITCH_WIDTH / 100
    )

    return angle(left, right)

In [ ]:
df_events['shotWindow'] = df_events.apply(
    lambda row: shot_window(row['xPosStart'], row['yPosStart']),
    axis=1
)

##  **Feature engineering** - Processing Tags

If applicable, we can extract the information with which bodypart the event was carried out and if it was the strong/weak foot by checking the player-data.

In [ ]:
def label_bodyPart(tags: list, strong_foot: str) -> str:
    relevant_tags = [401, 402, 403]
    filtered_tags = list(set(tags) & set(relevant_tags))
    if len(filtered_tags) > 0:
        bodyPart = df_tag_labels.loc[filtered_tags[0], 'Label'].lower()
        if bodyPart == 'head/body':
            label = bodyPart
        elif bodyPart == strong_foot:
            label = 'strong foot'
        else:
            label = 'weak foot'
    else:
        label = 'unknown'
    return label

In [ ]:
df_events['bodyPart'] = df_events.apply(
    lambda row: label_bodyPart(
        row['tags'],
        df_players.loc[row['playerId'], 'foot']
    ),
    axis=1
)

For each event we extract the role the player plays in (goalkeeper, defender, midfielder or forward).

In [ ]:
df_events['playerRole'] = df_events.apply(
    lambda row: df_players.loc[row['playerId'], 'role.code3'],
    axis=1
)

For shot-events, we can extract information about which part of the goal the shot was aimed at from the tags.

In [ ]:
def label_tags(tags: list, relevant_tags: list) -> str:
    filtered_tags = list(set(tags) & set(relevant_tags))
    if len(filtered_tags) > 0:
        label = df_tag_labels.loc[filtered_tags[0], 'Label'].lower()
    else:
        label = 'unknown'
    return label

In [ ]:
df_events['aimedGoalPart'] = df_events['tags'].apply(
    lambda x: label_tags(x, list(range(1201, 1224)))
)

Using the tag-mapping, we can create some more tag-flags.

In [ ]:
tag_mappings = {
    'keyPass': 302,
    'dangerousBallLost': 2001,
    'opportunity': 201,
    'counterAttack': 1901,
    'through': 901,
    'high': 801
}

for col, tag_id in tag_mappings.items():
    df_events[col] = df_events['tags'].apply(
        lambda x: tag_id in x
    )

df_events['accurate'] = df_events['tags'].apply(
    lambda x: True if 1801 in x else False if 1802 in x else None
)

df_events['goal'] = df_events['tags'].apply(
    lambda x: 101 in x or 102 in x
)

We finish the feature-engineering part by providing some information about the previous event. It might be interesting for shot-analysis later on to see the effect of the type of the preceding event (e.g. high/low pass or dribble) on the shot's outcome.  
For simplicity we just shift the columns of interest by one and disregard the edge cases (each match-half's first and last event), as they are not interesting for our purpose.

In [ ]:
cols_shift = [
    'eventName', 'distance', 'angle', 'angleCategory',
    'dangerousBallLost', 'keyPass', 'through', 'high'
]

df_events[
    [f'prev{col[0].capitalize()}{col[1:]}' for col in cols_shift]
] = df_events[cols_shift].shift(1)

## **Pass analysis** - Comparison of passing-profiles

Let's compare all the players' passing-skills based on 3 features: average passing-accuracy, forward rate and share of passes into the final third.

In [ ]:
pass_stats = df_events[
    df_events['eventName'] == 'Pass'
].groupby('playerId').agg({
    'id': 'count',
    'accurate': 'mean',
    'angleCategory': lambda x: (
        (x == 'diagonal forward') |
        (x == 'vertical forward')
    ).mean(),
    'intoFinalThird': 'mean'
}).rename(
    columns={
        'accurate': 'passingAccuracy',
        'angleCategory': 'forwardRate',
        'id': 'passCount'
    }
)
overlap_cols = set(df_players.columns) & set(pass_stats.columns)
df_players = df_players.drop(columns=overlap_cols)
df_players = df_players.join(
    pass_stats,
    how='left'
)

In [ ]:
relevant_passers = df_players[df_players['passCount'] >= 250]
sns.scatterplot(
    data=relevant_passers,
    x='passingAccuracy',
    y='forwardRate',
    hue='role.code2',
    s=20,
    edgecolor='black',
    linewidth=0.5
)
sns.despine(right=True, top=True)
plt.title(
    'Player\'s passing-accuracy and forward-rate'
)
plt.xlabel('passing accuracy')
plt.ylabel('forward rate')
plt.legend(title='Position')
plt.gca().xaxis.set_major_formatter(PercentFormatter(1))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))

As we can see, the passing-profile highly depends on the player's position: The average passing-accuracy and forward-rate decreases with the player's position being more forward on the pitch, as there is more direct pressure from opponents and less opportunities to find a forward-pass.

In [ ]:
top_tier_passers = df_players[
    (df_players['passCount'] >= 1000) &
    (df_players['role.code2'] != 'GK') &
    (df_players['passingAccuracy'] >= 0.9) &
    (df_players['forwardRate'] >= 0.5)
]
print(top_tier_passers['role.code2'].value_counts())

Unsurprisingly, the top-tier passers (excluding goalkeepers) are predominantly defenders.

Next, we examine the top-tier passers in greater detail, focusing on field players with both high passing accuracy and forward rate. Within this elite group, we identify the top performers for each passing metric.

In [ ]:
sns.scatterplot(
    data=top_tier_passers,
    x='passingAccuracy',
    y='forwardRate',
    hue='intoFinalThird',
    edgecolor='black',
    linewidth=1
)
for col in ['passingAccuracy', 'forwardRate', 'intoFinalThird']:
    player_id = top_tier_passers[col].idxmax()
    player_name = df_players.loc[player_id, 'shortName']
    player_role = df_players.loc[player_id, 'role.code2']
    pass_count = df_players.loc[player_id, 'passCount']
    intoFinalThird = df_players.loc[player_id, 'intoFinalThird']
    passing_accuracy = df_players.loc[player_id, 'passingAccuracy']
    forward_rate = df_players.loc[player_id, 'forwardRate']
    annotation_text = (
        f'{player_name} ({player_role})\n'
        f'pass count: {int(pass_count)}\n'
        f'passing accuracy: {passing_accuracy * 100:.2f}%\n'
        f'forward rate: {forward_rate * 100:.2f}%\n'
        f'into final third: {intoFinalThird * 100:.2f}%\n'
    )
    plt.annotate(
        annotation_text,
        (passing_accuracy + 0.001, forward_rate + 0.001),
        fontsize=8,
        va='top',
        ha='left'
    )
sns.despine(right=True, top=True)
plt.xlim(0.895, 1)
plt.ylim(0.475, 0.675)
plt.gca().xaxis.set_major_formatter(PercentFormatter(1))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title(
    'Top passers based on accuracy, forward-rate\n' +
    'and share of passes into the final third'
)
plt.xlabel('passing accuracy')
plt.ylabel('forward rate')
handles, labels = plt.gca().get_legend_handles_labels()
percent_labels = [f'{float(label)*100:.1f}%'for label in labels]
plt.legend(
    handles,
    percent_labels,
    title='share of passes\ninto final third'
)
plt.show()

Based on the criteria of passing accuracy, forward rate and share of passes into the final third, the three best passers from the top tier are Sergio Ramos, John Stones and Toni Kroos. Notably, Toni Kroos stands out as one of only 4 midfielders in this top-tier category.

## **Pass analysis** - Elite passers visualizations and statistics

To get a better idea of these players' passing-profiles, let's take a look at all of their passes in one of their matches, starting with Sergio Ramos.

In [ ]:
vis.plot_passes(
    df_events,
    df_players,
    df_matches,
    3306,
    2565927,
    'angleCategory'
)

As we can see, Ramos played as a left center-back in that game. Based on his passes, he stayed in his defensive position most of the game, with only a few passes in the opponent's half.

Next, we examine Toni Kroos against Celta de Vigo.

In [ ]:
vis.plot_passes(
    df_events,
    df_players,
    df_matches,
    14723,
    2565912,
    'angleCategory'
)

Typical for Toni Kroos, he is all over the pitch, picking up the ball in defense as well as playing from a higher, mostly central position. As we can see, he is distributing the ball very well to both sides with mostly diagonal forward passes and an extraordinarily high accuracy, even in the opponent's half.

The next deep dive is into John Stones' match against Newcastle.

In [ ]:
vis.plot_passes(
    df_events,
    df_players,
    df_matches,
    9380,
    2499954,
    'angleCategory'
)

Even though John Stones played as a center-back in his first seasons under Pep Guardiola, we can see that he was positioned exceptionally high up the pitch, reflecting Manchester City's tactical approach of controlling games through possession and territorial dominance. Opponents were frequently pinned in their own half, with City systematically recycling possession and methodically probing for openings.

Let's now take a look at the comprehensive passing statistics for all three players across the entire season, beginning with Sergio Ramos.

In [ ]:
vis.plot_passing_stats(df_events, df_players, df_matches, 3306)

As we saw earlier when comparing the top-tier-passers, Ramos has an outstanding forward rate, playing over 60% of his passes forward while still maintaining a high accuracy.

Let's now have a look a Toni Kroos.

In [ ]:
vis.plot_passing_stats(df_events, df_players, df_matches, 14723)

With nearly 2900 passes, Kroos played the most passes out of the three players. This makes his high forward rate and passing accuracy stand out even more, especially when consider that as a central midfielder, he is under significantly more opponent-pressure as he is playing higher up the pitch.

We conclude the passing-analysis by looking at John Stones' stats.

In [ ]:
vis.plot_passing_stats(df_events, df_players, df_matches, 9380)

Stones completed fewer passes than Ramos and Kroos while registering the highest proportion of horizontal passes among the three players. Nevertheless, his overall passing accuracy of nearly 97% is extraordinary.

## **Shot analysis** - Handling Outliers

The objective of this section is to train a logistic regression model on all shots contained in df_events (including free-kick shots and penalties) to predict shot outcomes.
To accomplish this, we first create a new dataframe containing only shot events.

In [ ]:
df_shots = df_events[df_events['eventName'] == 'Shot'].copy()
df_freekicks = df_events[df_events['subEventName'] == 'Free kick shot'].copy()
df_penalties = df_events[df_events['subEventName'] == 'Penalty'].copy()

df_shots = pd.concat([df_shots, df_freekicks, df_penalties], axis=0)

Before proceeding with the shot analysis, we identify and address outliers that could negatively impact the model's performance.

Given the extremely small sample of shots taken by goalkeepers (only 7), we exclude these observations as they're not enough to derive meaningful statistical insights.

In [ ]:
goalkeeper_shots = df_shots[df_shots['playerRole'] == 'GKP']['subEventName']
df_shots.drop(goalkeeper_shots.index, inplace=True)
print(
    f'{goalkeeper_shots.shape[0]} goalkeeper shots removed from dataset.'
)

As seen in below histogram, there are only very few shots taken from a distance greater than 40 meters.

In [ ]:
ax = sns.histplot(x=df_shots['goalDistanceStart'], bins=10)
sns.despine(right=True, top=True)


As those shots are only interesting when we know the goalkeeper's position (and as we also see below none of those shots is preceded by the event 'Goalkeeper leaving line' so we have no information about the goalkeeper's position).

In [ ]:
max_distance_goalkeeper_out = df_shots[
    df_shots['prevEventName'] == 'Goalkeeper leaving line'
]['goalDistanceStart'].max()
print(
    f'Max shot distance with goalkeeper off line: '
    f'{max_distance_goalkeeper_out:.2f} m'
)

So for a better general model-performance we will drop all of the shots taken from very far out (> 40m).

In [ ]:
far_shots = df_shots[df_shots['goalDistanceStart'] > 40]
df_shots.drop(
    far_shots.index,
    inplace=True
)
print(
    f'{far_shots.shape[0]} far shots removed from dataset.'
)

As the goals wouldn't count even if the shot went in, we filter out shots from an offside-position.

In [ ]:
shots_after_offside = df_shots[
    df_shots['prevEventName'] == 'Offside'
]
df_shots.drop(
    shots_after_offside.index,
    inplace=True
)
print(
    f'{shots_after_offside.shape[0]} shots after offside removed from dataset.'
)

## **Shot analysis** - Single matches' shot-visualization

To explore a single matches' shots, we can plot the position the shots were taken from and add some information about a specified feature.

In [ ]:
vis.plot_shots(
    df_shots,
    df_players,
    df_teams,
    df_matches,
    'bodyPart',
    True,
    2516785,
    2447,
    25867
)

Looking at Pierre Aubameyang's performance against Borussia Mönchengladbach, we can see that he scored 3 out of his 4 shots taken with his strong foot, while his attempts with his weak foot and headers failed to find the target. The shots were all taken from inside the box. Notably, all shots were taken from positions within the penalty area, suggesting his positioning and movement were excellent.

## **Shot analysis** - Goal-probability regarding distance, angle and body part

To see how the body-part a shot is taken with affects the goal-probability, let's first visualize each of the feature's correlation to a shot's outcome.

In [ ]:
vis.feature_goal_corr_bars(df_shots, 'bodyPart', 5, False)

Counterintuitively, the initial analysis reveals that shots taken with the strong foot exhibit the lowest conversion probability, while head/body shots have the highest goal rate. However, examining the number of shots taken puts this into perspective: Strong foot attempts make out more than double the number of all other body parts combined, suggesting a potential selection bias in shot opportunity quality.

Another important factor could be that headers and weak foot shots may be predominantly taken from more advantageous positions — specifically closer distances and superior angles to goal.

Let's first see how distance and angle to goal affect goal-probability before investigating the shots' body-part distribution regarding distance and angle.

In [ ]:
vis.feature_goal_corr_bars(df_shots, 'goalDistanceStart', 4, False)

Excluding shots from extreme distances (>80m) due to their limited sample size and potential data quality issues, we , we can clearly see that goal-rate decrease with increasing distance.

In addition to distance, shooting angle represents a critical factor in shot outcome analysis.

In [ ]:
vis.feature_goal_corr_bars(df_shots, 'shotWindow', 8, False)

The analysis reveals that goal conversion rates have a positive correlation with shot-window-angle, indicating that superior shooting angles yield higher scoring probabilities.

Regarding these insights, let's inspect the bodyparts' shots' distance- and angle-distribution.

In [ ]:
vis.category_heatmap(df_shots, 'goalDistanceStart', 'bodyPart', 4, 3)

In [ ]:
vis.category_heatmap(df_shots, 'shotWindow', 'bodyPart', 8, 3)

As assumed, the distances and angles shots are taken from are very different for each bodypart.

About 15% of the shots are headers/bodyshots, two in three of them were taken from a very close distance of less than 10m.
Looking at the shot-windows, about 9 in 10 headers/bodyshots had a good shot-windos of more than 22.5 degrees

A little more shots in total are taken with the weak foot, half of them from closer than 20m from the goal, the rest from a range between 20m and 30m. Almost three in five shots taken with the weak foot had a narrow shot-window of less than 15 degrees.

With about 66% in total, the biggest share of shots are taken with the strong foot, almost half of them from a from a distance of more than 20m. Still around 5% of all shots are taken with the strong foot from more than 30m of distance to the goal. The shot's angles resemble the angles of weak-foot-shots with almost 3 out of 4 shots were taken from a bad angle, having a narrow shot-window of less than 22.5 degrees.

With this information, the bodyparts' goal-ratios don't seem that suprising anymore as the strong-foot-shots are mostly taken from worse distances and angles. So let's compare the bodyparts-goal-rates separately in each of the distance-/angle-categories.

In [ ]:
vis.two_features_goal_corr_bars(
    df_shots,
    'goalDistanceStart',
    'bodyPart',
    4,
    3,
    False
)

The strong foot has the (shared) highest goal-rate in each of the distance-categories (disregarding range from 20m to 30m where headers have a higher goal-rate due to the very small sample size).

In [ ]:
vis.two_features_goal_corr_bars(
    df_shots,
    'shotWindow',
    'bodyPart',
    8,
    3,
    False
)


Strong foot shots achieve the (shared) highest conversion rates in 6 of the 8 shot-window categories examined.

**Conclusion**: At first glance, the overall goal rates surpisingly suggested that the probability of scoring a goal with the head/body or the weak foot were higher than with the strong foot. However, diving deeper and analysing the shots' distance- and angle-distribution showed that this was due to the fact that strong-foot-shots tended to be taken from a higher distance and a worse angle than those with the weak foot or head/body. Eliminating this factor and comparing the bodyparts' goal-rates in each separate distance-/angle-category, we can conclude that the strong foot has the highest specific goal-rate, followed by the weak foot shots and headers/bodyshots.

## **Prediction of Outcome** - xGoal-classification with Logistic Regression

Let's go ahead and create an xGoal-model. We will train and test the model using all of the leagues except the Bundesliga and then apply it on the Bundesliga to do some analysis using the xGoals.

First of all we define the relevant features we will use for the regression according to our previous analysis.

In [ ]:
relevant_cols_dict = {
    'numerical': ['goalDistanceStart', 'shotWindow'],
    'boolean': [
        'prevDangerousBallLost', 'counterAttack',
        'prevThrough', 'prevHigh'
    ],
    'categorical': [
        'subEventName', 'bodyPart', 'prevEventName',
        'playerRole', 'competition'
    ]
}

relevant_cols = (
    relevant_cols_dict['numerical'] +
    relevant_cols_dict['boolean'] +
    relevant_cols_dict['categorical']
)

X = df_shots.loc[:, relevant_cols].copy()

y = df_shots['goal'].copy()

We seperate the bundesliga-data from the rest so we can later apply the trained model on it.

In [ ]:
df_bundesliga = df_shots[
    df_shots['competition'] == 'germany'
].copy()

X.drop(df_bundesliga.index, inplace=True)
y.drop(df_bundesliga.index, inplace=True)

X.drop(columns=['competition'], inplace=True)
relevant_cols_dict['categorical'].remove('competition')
df_bundesliga.drop(columns=['competition'], inplace=True)

We encode the categorical features using one-hot encoding.

In [ ]:
ct = ColumnTransformer(
    transformers=[
        ('', OneHotEncoder(drop='first'), relevant_cols_dict['categorical'])
    ],
    remainder='passthrough'
)
X = ct.fit_transform(X)

columns_transformed = ct.get_feature_names_out()

X = pd.DataFrame(X, columns=columns_transformed)

Perform a gridsearch to find the best hyperparameters (may outcomment the big grid to save time)

In [ ]:
model = LogisticRegression(random_state=42)

'''param_grid = {
    'C': np.logspace(-4, 4, 10),
    'class_weight': ['balanced', None],
    'max_iter': [250, 500, 750],
    'penalty': ['l1', 'l2', None],
    'solver': ['liblinear', 'newton-cholesky', 'saga']
}'''

# Fix the best performing hyperparameters to save computation time
param_grid = {
    'C': [2.782559402207126],
    'class_weight': ['balanced'],
    'max_iter': [750],
    'penalty': ['l2'],
    'solver': ['saga']
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X, y)

print('Best Hyperparameters:', grid_search.best_params_)
print('Best f1-score:', grid_search.best_score_)

Let's take a look at the fitted models' coefficients to better understand what features are important when predicting a shot's outcome

In [ ]:
best_model = grid_search.best_estimator_
best_model.fit(X, y)

feature_importance = pd.DataFrame(
    {'Feature': X.columns, 'Coefficient': best_model.coef_[0]}
)

feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(
    by='Abs_Coefficient',
    ascending=False
)

feature_importance[['Feature', 'Coefficient']]

Apart from the obvious fact that penalties have a higher goal-probability than other shots, the fitted model's coefficients show, that the type of event (such as the opponent losing the ball in a dangerous position or the goalkeeper saving a shot) preceding the shot has a big impact on it's outcome, followed by the bodypart it was taken with.

## **Prediction of Outcome** - Applying the model on the Bundesliga

To analyse our traind model's performance, we apply it on the bundesliga-data.

In [ ]:
df_bundesliga_transformed = ct.transform(df_bundesliga)
df_bundesliga_transformed = pd.DataFrame(
    df_bundesliga_transformed,
    columns=columns_transformed
)

In [ ]:
y_test = df_shots[df_shots['competition'] == 'germany']['goal']
y_pred = best_model.predict(df_bundesliga_transformed)
y_pred_probability = best_model.predict_proba(df_bundesliga_transformed)[:, 1]

print(f'F1 score: {f1_score(y_test, y_pred):.4f}')

Let's take a look at the prediction-outcome's confusion-matrix. Besides the classes' sizes, let's also consider the xGoals-average (i.e. how 'sure' the model was of the predicted outcome)

In [ ]:
def conf_matrix_avg_probability(
    y_test: pd.Series,
    y_pred: pd.Series,
    y_pred_proba: pd.Series
) -> None:
    conf_matrix = confusion_matrix(y_test, y_pred)

    df = pd.DataFrame({
        'actual': y_test,
        'predicted': y_pred,
        'probability': y_pred_proba
    })

    df_TN = df[~df['actual'] & ~df['predicted']]
    df_FP = df[~df['actual'] & df['predicted']]
    df_FN = df[df['actual'] & ~df['predicted']]
    df_TP = df[df['actual'] & df['predicted']]

    xgoals_avg_annot = [
        [
            f' / avg xG: {df_TN['probability'].mean():.2f}',
            f' / avg xG: {df_FP['probability'].mean():.2f}'
        ],
        [
            f' / avg xG: {df_FN['probability'].mean():.2f}',
            f' / avg xG: {df_TP['probability'].mean():.2f}'
        ]
    ]

    conf_matrix_annot = np.char.add(
        conf_matrix.astype(str),
        xgoals_avg_annot
    )

    ax = sns.heatmap(
        data=confusion_matrix(y_test, y_pred),
        annot=conf_matrix_annot,
        cbar=False,
        cmap='coolwarm',
        fmt='',
        square=True
    )

    ax.set_xticklabels(['Predicted Negative', 'Predicted Positive'])
    ax.set_yticklabels(['Actual Negative', 'Actual Positive'])
    plt.title('Confusion matrix with average xG')
    return None

In [ ]:
conf_matrix_avg_probability(y_test, y_pred, y_pred_probability)

As we can see, the model on average was close to predict the false negatives correctly as positives and the false positives correctly as negatives.

Let's investigate on what the table would have looked like at the end of the season following the xgoals.

In [ ]:
df_matches_bundesliga = df_matches[
    df_matches['competition'] == 'germany'
].copy()

df_bundesliga['xG'] = y_pred_probability
xg_per_match = df_bundesliga.groupby(
    ['matchId', 'teamId']
)['xG'].sum()

First we have to compare each matches' xgoals to get the xWinner. We consider a xdraw-tolerance, meaning that xg-differences below a specified tolerance count as a draw.

In [ ]:
xg_max = xg_per_match.groupby('matchId').max()
xg_min = xg_per_match.groupby('matchId').min()
xg_diff = xg_max - xg_min

x_winner = xg_per_match.groupby('matchId').idxmax().apply(lambda x: x[1])
x_winner = x_winner[xg_diff >= co.XDRAW_TOLERANCE]

df_matches_bundesliga['xWinner'] = (
    df_matches_bundesliga.index.to_series().apply(
        lambda x: x_winner[x] if x in x_winner else 0
    )
)

We now count the xwinners and xdraws to calculate each team's xpoints.

In [ ]:
df_xwins = df_matches_bundesliga[
    df_matches_bundesliga['xWinner'] != 0
].copy()
xwin_points = df_xwins['xWinner'].value_counts() * 3

In [ ]:
df_xdraws = df_matches_bundesliga[df_matches_bundesliga['xWinner'] == 0].copy()
df_xdraws['teamIds'] = df_xdraws['teamsData'].apply(
    lambda data: list(data.keys())
)
df_xdraws = df_xdraws.explode('teamIds')
xdraw_points = df_xdraws['teamIds'].astype(int).value_counts()

Additionally we count the total scored/conceded xGoals per team.

In [ ]:
def xgoals_conceded(teamId: int) -> float:
    matches = df_bundesliga[
        df_bundesliga['teamId'] == teamId
    ]['matchId'].unique()
    df_matches_events = df_bundesliga[
        df_bundesliga['matchId'].isin(matches)
    ]
    df_opponent_events = df_matches_events[
        df_matches_events['teamId'] != teamId
    ]
    return df_opponent_events['xG'].sum().round(2)

In [ ]:
total_xgoals = df_bundesliga.groupby('teamId')['xG'].sum().round(2)
total_xgoals_conceded = {
    teamId: xgoals_conceded(teamId)
    for teamId in df_bundesliga['teamId'].unique()
}

We calculate each team's actual points using the information about the matches' outcomes from the real data.

In [ ]:
df_wins = df_matches_bundesliga[df_matches_bundesliga['winner'] != 0].copy()
win_points = df_wins['winner'].value_counts() * 3

In [ ]:
df_draws = df_matches_bundesliga[df_matches_bundesliga['winner'] == 0].copy()
df_draws['teamIds'] = df_draws['teamsData'].apply(
    lambda data: list(data.keys())
)
df_draws = df_draws.explode('teamIds')
draw_points = df_draws['teamIds'].astype(int).value_counts()

To complete the data we need for our final table, we count each team's actual scored and conceded goals.

In [ ]:
def goals_scored(teamId: int) -> int:
    matches = df_bundesliga[
        df_bundesliga['teamId'] == teamId
    ]['matchId'].unique()
    goals = [
        df_matches_bundesliga.loc[match, 'teamsData'][str(teamId)]['score']
        for match in matches
    ]
    return sum(goals)

In [ ]:
def goals_conceded(teamId: int) -> int:
    matches = df_bundesliga[
        df_bundesliga['teamId'] == teamId
    ]['matchId'].unique()
    goals_conceded = [
        df_matches_bundesliga.loc[match, 'teamsData'][
            ut.get_other_value(
                list(df_matches_bundesliga.loc[match, 'teamsData'].keys()),
                str(teamId)
            )
        ]['score']
        for match in matches
    ]
    return sum(goals_conceded)

In [ ]:
total_goals = {
    teamId: goals_scored(teamId)
    for teamId in df_bundesliga['teamId'].unique()
}
total_goals_conceded = {
    teamId: goals_conceded(teamId)
    for teamId in df_bundesliga['teamId'].unique()
}

We now have all the information available and can start assembling our final table by putting together the actual data

In [ ]:
table = pd.DataFrame({'teamId': df_bundesliga['teamId'].unique()})
table['name'] = table['teamId'].apply(lambda id: df_teams.loc[id, 'name'])
table['points'] = table['teamId'].apply(
    lambda id: win_points.get(id, 0) + draw_points.get(id, 0)
)
table['goals'] = table['teamId'].apply(lambda id: total_goals.get(id, 0))
table['goalsConceded'] = table['teamId'].apply(
    lambda id: total_goals_conceded.get(id, 0)
)
table['goalDiff'] = table['goals'] - table['goalsConceded']


To compare the actual with the expected outcome, we add the expected data.

In [ ]:
table['xPoints'] = table['teamId'].apply(
    lambda id: xwin_points.get(id, 0) + xdraw_points.get(id, 0)
)
table['xG'] = table['teamId'].apply(lambda id: total_xgoals.get(id, 0))
table['xGConceded'] = table['teamId'].apply(
    lambda id: total_xgoals_conceded.get(id, 0)
)
table['xGDiff'] = table['xG'] - table['xGConceded']
table['points_diff'] = table['points'] - table['xPoints']
table.set_index('teamId', inplace=True)

We sort the teams in the leagute table by deducting their rank and xRank from points/xPoints.

In [ ]:
for col, diff in [('points', 'goalDiff'), ('xPoints', 'xGDiff')]:
    table_sorted = table.sort_values(diff, ascending=False)
    table[f'{col}_rank'] = table_sorted[col].rank(
        ascending=False,
        method='first'
    ).astype(int)
table['rank_diff'] = table['xPoints_rank'] - table['points_rank']

table.sort_values(by='points_rank', ascending=True, inplace=True)
general_cols = ['name', 'points_diff', 'rank_diff']
actual_cols = ['points_rank', 'points', 'goals', 'goalsConceded', 'goalDiff']
expected_cols = ['xPoints_rank', 'xPoints', 'xG', 'xGConceded', 'xGDiff']
table[general_cols + actual_cols + expected_cols]

Let's visualize the actual/expected table's comparison to see which teams over-/underperformed.

In [ ]:
plt.plot(
    [-1, 19],
    [-1, 19],
    'k--',
    alpha=0.25
)
plt.grid(True, alpha=0.3)

scatter = plt.scatter(
    table['xPoints_rank'],
    table['points_rank'],
    c=table['points_diff'],
    cmap='RdYlGn',
    s=100,
    edgecolors='black',
    linewidths=1,
    zorder=5
)

for idx, row in table.iterrows():
    plt.annotate(
        row['name'],
        (row['xPoints_rank'], row['points_rank']),
        xytext=(7, 0),
        textcoords='offset points',
        fontsize=8,
        ha='left',
        zorder=6
    )

plt.colorbar(
    scatter,
    label='Points Difference (xPoints - points)',
    orientation='vertical',
    shrink=0.6
)
plt.xlabel('xPoints rank')
plt.ylabel('points rank')
plt.xlim(0, 19)
plt.ylim(0, 19)
plt.xticks(range(1, 19))
plt.yticks(range(1, 19))
plt.gca().invert_yaxis()
plt.show()

**Key Insights**
- Bayern Munich demonstrated league dominance, maintaining the top position in both expected and actual performance metrics
- Upper table dynamics: Borussia Dortmund and Bayer Leverkusen underperformed relative to their expected outcomes, being surpassed by TSG Hoffenheim and FC Schalke 04, who both significantly exceeded their projected performance levels
- Lower table analysis: Hertha BSC Berlin outperformed their expected league position, successfully avoiding relegation through superior conversion of scoring opportunities